# Bayesian Analysis

Coco Yu

### Set Up Environment

Set up parameters as variables

In [ ]:
(study <- params$study)


[1] "ema"

[1] "1day"

[1] 0

[1] "v5"

[1] "nested"

[1] "main"

Function conflicts

In [ ]:

# source
devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/fun_ml.R?raw=true")


ℹ SHA-1 hash of file is "77e91675366f10788c6bcb59fa1cfc9ee0c75281"

Chunk Defaults

In [ ]:

knitr::opts_chunk$set(attr.output='style="max-height: 500px;"')

options(tibble.width = Inf)
options(tibble.print_max = Inf)


Packages for script

In [ ]:

library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──
✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     
── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use tidymodels_prefer() to resolve common conflicts.

Source support functions

In [ ]:

devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")


ℹ SHA-1 hash of file is "a58e57da996d1b70bb9a5b58241325d6fd78890f"

Absolute paths

In [ ]:
path_fairema <- format_path(str_c("studydata/risk/data_processed/fairema"))


### Bayesian Analysis

In [ ]:
contrast_table <- function(df, met, levels){
  df |> 
    filter(group != "all") |>
    select(c(met, "group", "outer_split_num")) |> 
    group_by(group) |>
    mutate(repeat_num = trunc((outer_split_num - 1) / 10 + 1),
           fold_num = outer_split_num %% 10) |> 
    select(c(met, "group", "repeat_num", "fold_num")) |>
    pivot_wider(values_from = c(met), names_from = group) |>
    rename(id = repeat_num, id2 = fold_num) |>
    perf_mod(formula = statistic ~ model + (1 | id2/id),
             # prior_intercept = rstanarm::student_t(autoscale = TRUE),
             # prior = rstanarm::student_t(autoscale = TRUE),
             # transform = tidyposterior::logit_trans,  # for skewed & bounded AUC
             # iter = 2000, chains = 4,
             adapt_delta = .99,
             # cores = 4, seed = 12345,
             family = gaussian, verbose = FALSE) |> 
    contrast_models(list_1 = levels[1], list_2 = levels[2]) |> 
    summary(size = 0) |> 
    mutate(metrics = met, .before = contrast)
  }


In [ ]:
df_sex <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "sex", 
                                                  "_", window, "_", lead, "_", 
                                                  version, "_", cv, "_", model,
                                                  ".csv")),
                      col_types = cols()) |> 
  filter(group != "all", n_lapses != 0) |> 
  select(-n_lapses)

rbind(
  contrast_table(df_sex, "auroc", c("Male", "Female")),
  contrast_table(df_sex, "prauc", c("Male", "Female")),
  contrast_table(df_sex, "sensitivity", c("Male", "Female")),
  contrast_table(df_sex, "specificity", c("Male", "Female")),
  contrast_table(df_sex, "ppv", c("Male", "Female")),
  contrast_table(df_sex, "balaccuracy", c("Male", "Female"))
)


ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(met)

  # Now:
  data %>% select(all_of(met))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.


SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 4.7e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.47 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.777 seconds (Warm-up)
Chain 1:                0.208 seconds (Sampling)
Chain 1:                0.98

# A tibble: 6 × 10
  metrics     contrast       probability     mean     lower  upper  size
  <chr>       <chr>                <dbl>    <dbl>     <dbl>  <dbl> <dbl>
1 auroc       Male vs Female       0.990  0.0439   0.0138   0.0744     0
2 prauc       Male vs Female       0.572  0.0115  -0.0801   0.106      0
3 sensitivity Male vs Female       0.660  0.0210  -0.0645   0.106      0
4 specificity Male vs Female       0.981  0.0564   0.0113   0.101      0
5 ppv         Male vs Female       0.430 -0.00649 -0.0706   0.0570     0
6 balaccuracy Male vs Female       0.946  0.0388  -0.000731 0.0781     0
  pract_neg pract_equiv pract_pos
      <dbl>       <dbl>     <dbl>
1        NA          NA        NA
2        NA          NA        NA
3        NA          NA        NA
4        NA          NA        NA
5        NA          NA        NA
6        NA          NA        NA

In [ ]:
df_race <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "race", 
                                                   "_", window, "_", lead, "_", 
                                                   version, "_", cv, "_", model,
                                                   ".csv")),
                    col_types = cols()) |> 
  filter(group != "all", n_lapses != 0) |> 
  select(-n_lapses)

rbind(
  contrast_table(df_race, "auroc", c("white", "non_white")),
  contrast_table(df_race, "prauc", c("white", "non_white")),
  contrast_table(df_race, "sensitivity", c("white", "non_white")),
  contrast_table(df_race, "specificity", c("white", "non_white")),
  contrast_table(df_race, "ppv", c("white", "non_white")),
  contrast_table(df_race, "balaccuracy", c("white", "non_white"))
)



SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 2.2e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.22 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.399 seconds (Warm-up)
Chain 1:                0.392 seconds (Sampling)
Chain 1:                0.79

# A tibble: 6 × 10
  metrics     contrast           probability   mean  lower upper  size pract_neg
  <chr>       <chr>                    <dbl>  <dbl>  <dbl> <dbl> <dbl>     <dbl>
1 auroc       white vs non_white       1     0.193  0.127  0.259     0        NA
2 prauc       white vs non_white       1.00  0.223  0.121  0.324     0        NA
3 sensitivity white vs non_white       0.999 0.246  0.132  0.363     0        NA
4 specificity white vs non_white       0.967 0.0808 0.0103 0.152     0        NA
5 ppv         white vs non_white       0.984 0.103  0.0255 0.182     0        NA
6 balaccuracy white vs non_white       1     0.163  0.121  0.206     0        NA
  pract_equiv pract_pos
        <dbl>     <dbl>
1          NA        NA
2          NA        NA
3          NA        NA
4          NA        NA
5          NA        NA
6          NA        NA

In [ ]:
df_income <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", 
                                                     "income", "_", window, "_", 
                                                     lead, "_", version, "_", cv, 
                                                     "_", model, ".csv")),
                      col_types = cols()) |> 
  filter(group != "all", n_lapses != 0) |> 
  select(-n_lapses)

rbind(
  contrast_table(df_income, "auroc", c("above_poverty", "below_poverty")),
  contrast_table(df_income, "prauc", c("above_poverty", "below_poverty")),
  contrast_table(df_income, "sensitivity", c("above_poverty", "below_poverty")),
  contrast_table(df_income, "specificity", c("above_poverty", "below_poverty")),
  contrast_table(df_income, "ppv", c("above_poverty", "below_poverty")),
  contrast_table(df_income, "balaccuracy", c("above_poverty", "below_poverty"))
)



SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 2.6e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.26 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.387 seconds (Warm-up)
Chain 1:                0.239 seconds (Sampling)
Chain 1:                0.62

# A tibble: 6 × 10
  metrics     contrast                       probability     mean   lower
  <chr>       <chr>                                <dbl>    <dbl>   <dbl>
1 auroc       above_poverty vs below_poverty      0.998   0.131    0.0674
2 prauc       above_poverty vs below_poverty      0.968   0.123    0.0161
3 sensitivity above_poverty vs below_poverty      1       0.243    0.140 
4 specificity above_poverty vs below_poverty      0.0262 -0.0553  -0.103 
5 ppv         above_poverty vs below_poverty      0.532   0.00415 -0.0792
6 balaccuracy above_poverty vs below_poverty      0.999   0.0939   0.0468
     upper  size pract_neg pract_equiv pract_pos
     <dbl> <dbl>     <dbl>       <dbl>     <dbl>
1  0.196       0        NA          NA        NA
2  0.227       0        NA          NA        NA
3  0.344       0        NA          NA        NA
4 -0.00832     0        NA          NA        NA
5  0.0890      0        NA          NA        NA
6  0.139       0        NA          NA        

In [ ]:
df_age <- read_csv(here::here(path_fairema, str_c("outer_preds_perf_", "age", "_", window, 
                                           "_", lead, "_", version, "_", cv, "_", model,
                                           ".csv")),
                   col_types = cols()) |> 
  filter(group != "all", n_lapses != 0) |> 
  select(-n_lapses)

rbind(
  contrast_table(df_age, "auroc", c("younger", "older")),
  contrast_table(df_age, "prauc", c("younger", "older")),
  contrast_table(df_age, "sensitivity", c("younger", "older")),
  contrast_table(df_age, "specificity", c("younger", "older")),
  contrast_table(df_age, "ppv", c("younger", "older")),
  contrast_table(df_age, "balaccuracy", c("younger", "older"))
)



SAMPLING FOR MODEL 'continuous' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 2.1e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.21 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.785 seconds (Warm-up)
Chain 1:                0.203 seconds (Sampling)
Chain 1:                0.98

# A tibble: 6 × 10
  metrics     contrast         probability   mean    lower upper  size pract_neg
  <chr>       <chr>                  <dbl>  <dbl>    <dbl> <dbl> <dbl>     <dbl>
1 auroc       younger vs older       1     0.0979  0.0572  0.138     0        NA
2 prauc       younger vs older       0.919 0.0864 -0.0171  0.187     0        NA
3 sensitivity younger vs older       0.994 0.153   0.0633  0.240     0        NA
4 specificity younger vs older       0.966 0.0568  0.00612 0.106     0        NA
5 ppv         younger vs older       0.974 0.0764  0.0111  0.143     0        NA
6 balaccuracy younger vs older       1     0.107   0.0645  0.149     0        NA
  pract_equiv pract_pos
        <dbl>     <dbl>
1          NA        NA
2          NA        NA
3          NA        NA
4          NA        NA
5          NA        NA
6          NA        NA